In [1]:
import ast
import pandas as pd
from transformers import TapasTokenizer, TapasConfig, TapasForQuestionAnswering

In [2]:
config = TapasConfig.from_pretrained("google/tapas-base-finetuned-wtq")
model = TapasForQuestionAnswering.from_pretrained("google/tapas-base-finetuned-wtq", config=config)

tokenizer = TapasTokenizer.from_pretrained("google/tapas-base-finetuned-wtq")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
data = {
    "Titles": ["Glorious visions in animation and performance","Medicine for the 99 percent","What is so special about the human brain?","The mysterious science of pain","The uncomplicated truth about women's sexuality"],
    "Views": ["946000","310677","3082440","887739","2303625"],
    "Speakers": ["Miwa Matreyek","Thomas Pogge","Suzana Herculano-Houzel","Joshua Pate","Sarah Barmak"],
    "Duration": ["671","1085","811","287","680"],
    "Comments": ["148","121","1050","0","17"],
    "Events": ["TEDGlobal 2010","TEDxCanberra","TEDGlobal 2013","TED-Ed","TEDxToronto"]
}

queries = ["what is the title with more comments?","what is the event with the longest duration?", "How many speakers are there in the dataset?"]

table = pd.DataFrame.from_dict(data)
inputs = tokenizer(table=table, queries=queries, padding="max_length", return_tensors="pt")
outputs = model(**inputs)

predicted_answer_coordinates, predicted_aggregation_indices = tokenizer.convert_logits_to_predictions(
    inputs, outputs.logits.detach(), outputs.logits_aggregation.detach()
)

/usr/local/lib/python3.10/dist-packages/transformers/models/tapas/tokenization_tapas.py:2699: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/usr/local/lib/python3.10/dist-packages/transformers/models/tapas/tokenization_tapas.py:1493: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


**3 perguntas e 2 acertos**.

Na 3a pergunta ele acertou o agg, mas retornou a contagem de itens da lista respsota

In [4]:
id2aggregation = {0: "NONE", 1: "SUM", 2: "AVERAGE", 3: "COUNT"}
aggregation_predictions_string = [id2aggregation[x] for x in predicted_aggregation_indices]
answers = []

for coordinates in predicted_answer_coordinates:
    if len(coordinates) == 1:
        # only a single cell:
        answers.append(table.iat[coordinates[0]])
    else:
        # multiple cells
        cell_values = []

        for coordinate in coordinates:
            cell_values.append(table.iat[coordinate])

        answers.append(", ".join(cell_values))

for query, answer, predicted_agg in zip(queries, answers, aggregation_predictions_string):
    print(query)
    if predicted_agg == "NONE":
        print("Predicted answer: " + answer)
    else:
        print("Predicted answer: " + predicted_agg + " > " + answer)

what is the title with more comments?
Predicted answer: What is so special about the human brain?
what is the event with the longest duration?
Predicted answer: TEDxCanberra
How many speakers are there in the dataset?
Predicted answer: COUNT > Miwa Matreyek, Thomas Pogge, Suzana Herculano-Houzel, Joshua Pate, Sarah Barmak
